In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('jugadores_fantasy_migate.csv')

In [3]:
original_data = df[['Nombre', 'posicion']].copy()

In [4]:
df = pd.get_dummies(df, columns=['posicion', 'equipo'])


In [5]:
X = df.drop(['Nombre', 'Total_puntos', 'Precio'], axis=1)
y_puntos = df['Total_puntos']
y_precio = df['Precio']

In [6]:
X_train, X_test, y_puntos_train, y_puntos_test = train_test_split(X, y_puntos, test_size=0.3, random_state=42)
X_train, X_test, y_precio_train, y_precio_test = train_test_split(X, y_precio, test_size=0.3, random_state=42)


In [7]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [9]:
rf_puntos = RandomForestRegressor(random_state=42)
grid_search_puntos = GridSearchCV(estimator=rf_puntos, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search_puntos.fit(X_train_scaled, y_puntos_train)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.7s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.7s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.7s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   1.8s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estim

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             verbose=2)

In [10]:
best_rf_puntos = grid_search_puntos.best_estimator_

In [11]:
cv_scores_puntos = cross_val_score(best_rf_puntos, X_train_scaled, y_puntos_train, cv=5)
print("Puntos CV Scores:", cv_scores_puntos)
print("Puntos CV Mean Score:", cv_scores_puntos.mean())

Puntos CV Scores: [0.78826097 0.78094896 0.75792606 0.74904443 0.78518661]
Puntos CV Mean Score: 0.7722734051975526


In [12]:
rf_precio = RandomForestRegressor(random_state=42)
grid_search_precio = GridSearchCV(estimator=rf_precio, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search_precio.fit(X_train_scaled, y_precio_train)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.6s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.6s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.6s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   1.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   1.7s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.5s
[CV] END m

GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             verbose=2)

In [13]:
best_rf_precio = grid_search_precio.best_estimator_

In [14]:
cv_scores_precio = cross_val_score(best_rf_precio, X_train_scaled, y_precio_train, cv=5)
print("Precio CV Scores:", cv_scores_precio)
print("Precio CV Mean Score:", cv_scores_precio.mean())

Precio CV Scores: [0.637527   0.71327985 0.60993986 0.57326939 0.63081892]
Precio CV Mean Score: 0.6329670031382324


In [15]:
y_puntos_pred = best_rf_puntos.predict(X_test_scaled)
y_precio_pred = best_rf_precio.predict(X_test_scaled)

In [16]:
print("Puntos R2 Score:", best_rf_puntos.score(X_test_scaled, y_puntos_test))
print("Puntos MSE:", mean_squared_error(y_puntos_test, y_puntos_pred))
print("Precio R2 Score:", best_rf_precio.score(X_test_scaled, y_precio_test))
print("Precio MSE:", mean_squared_error(y_precio_test, y_precio_pred))

Puntos R2 Score: 0.8443609226503005
Puntos MSE: 207.88053544586054
Precio R2 Score: 0.7419164899407131
Precio MSE: 9865090804965.219


In [17]:
resultados = pd.DataFrame({
    'Nombre': original_data.loc[X_test.index, 'Nombre'],
    'Posicion': original_data.loc[X_test.index, 'posicion'],
    'Puntos_Predichos': y_puntos_pred,
    'Precio_Predicho': y_precio_pred
})

In [18]:
top_10_puntos = resultados.sort_values(by='Puntos_Predichos', ascending=False).head(10)
print("\n10 jugadores con mayor puntaje predicho:")
for i, row in top_10_puntos.iterrows():
    print(f"{row['Nombre']} - Puntos: {row['Puntos_Predichos']:.2f}, Precio: {row['Precio_Predicho']:.2f} €")

top_10_puntos = top_10_puntos.rename(columns={'Puntos_Predichos': 'Puntos', 'Precio_Predicho': 'Precio'})
top_10_puntos.to_csv('sugerencia.csv', index=False)

print("Datos exportados a sugerencia.csv")


10 jugadores con mayor puntaje predicho:
A. Griezmann - Puntos: 129.75, Precio: 17269536.47 €
J. Bellingham - Puntos: 124.86, Precio: 16379080.22 €
N. Williams - Puntos: 122.55, Precio: 17295089.78 €
A. Budimir - Puntos: 120.83, Precio: 16228322.64 €
V. Muriqi - Puntos: 113.88, Precio: 16331938.97 €
R. Lewandowski - Puntos: 88.65, Precio: 10467385.97 €
I. Aspas - Puntos: 87.52, Precio: 9770985.33 €
G. Moreno - Puntos: 87.29, Precio: 12394310.08 €
F. Valverde - Puntos: 86.77, Precio: 10909847.38 €
J. Cancelo - Puntos: 86.35, Precio: 10169154.45 €
Datos exportados a sugerencia.csv


In [19]:
top_10_precio = resultados.sort_values(by='Precio_Predicho', ascending=False).head(10)
print("\n10 jugadores con mayor precio predicho:")
for i, row in top_10_precio.iterrows():
    print(f"{row['Nombre']} - Puntos: {row['Puntos_Predichos']:.2f}, Precio: {row['Precio_Predicho']:.2f} €")


10 jugadores con mayor precio predicho:
N. Williams - Puntos: 122.55, Precio: 17295089.78 €
A. Griezmann - Puntos: 129.75, Precio: 17269536.47 €
J. Bellingham - Puntos: 124.86, Precio: 16379080.22 €
V. Muriqi - Puntos: 113.88, Precio: 16331938.97 €
A. Budimir - Puntos: 120.83, Precio: 16228322.64 €
G. Moreno - Puntos: 87.29, Precio: 12394310.08 €
T. Kroos - Puntos: 84.50, Precio: 11500140.77 €
M. Oyarzabal - Puntos: 82.31, Precio: 11412914.58 €
B. Méndez - Puntos: 74.97, Precio: 11229218.32 €
F. Valverde - Puntos: 86.77, Precio: 10909847.38 €


In [20]:
top_10_delanteros = resultados[resultados['Posicion'] == 'delantero'].sort_values(by='Puntos_Predichos', ascending=False).head(10)

print("\n10 delanteros con mayor puntaje predicho:")
for i, row in top_10_delanteros.iterrows():
    print(f"{row['Nombre']} - Puntos: {row['Puntos_Predichos']:.2f}, Posición: {row['Posicion']}, Precio: {row['Precio_Predicho']:.2f} €")

top_10_delanteros = top_10_delanteros.rename(columns={'Puntos_Predichos': 'Puntos', 'Precio_Predicho': 'Precio'})
top_10_delanteros.to_csv('sugerencia_delanteros.csv', index=False)
print("Datos de delanteros exportados a sugerencia_delanteros.csv")


10 delanteros con mayor puntaje predicho:
A. Griezmann - Puntos: 129.75, Posición: delantero, Precio: 17269536.47 €
A. Budimir - Puntos: 120.83, Posición: delantero, Precio: 16228322.64 €
V. Muriqi - Puntos: 113.88, Posición: delantero, Precio: 16331938.97 €
R. Lewandowski - Puntos: 88.65, Posición: delantero, Precio: 10467385.97 €
I. Aspas - Puntos: 87.52, Posición: delantero, Precio: 9770985.33 €
G. Moreno - Puntos: 87.29, Posición: delantero, Precio: 12394310.08 €
M. Oyarzabal - Puntos: 82.31, Posición: delantero, Precio: 11412914.58 €
J. Strand Larsen - Puntos: 76.58, Posición: delantero, Precio: 10501581.42 €
F. Torres - Puntos: 76.17, Posición: delantero, Precio: 8635450.85 €
S. Omorodion - Puntos: 68.48, Posición: delantero, Precio: 7073239.51 €
Datos de delanteros exportados a sugerencia_delanteros.csv


In [21]:
top_10_defensas = resultados[resultados['Posicion'] == 'defensa'].sort_values(by='Puntos_Predichos', ascending=False).head(10)

print("\n10 defensas con mayor puntaje predicho:")
for i, row in top_10_defensas.iterrows():
    print(f"{row['Nombre']} - Puntos: {row['Puntos_Predichos']:.2f}, Posición: {row['Posicion']}, Precio: {row['Precio_Predicho']:.2f} €")

top_10_defensas = top_10_defensas.rename(columns={'Puntos_Predichos': 'Puntos', 'Precio_Predicho': 'Precio'})
top_10_defensas.to_csv('sugerencia_defensas.csv', index=False)
print("Datos de defensas exportados a sugerencia_defensas.csv")


10 defensas con mayor puntaje predicho:
J. Cancelo - Puntos: 86.35, Posición: defensa, Precio: 10169154.45 €
Ó. De Marcos - Puntos: 86.18, Posición: defensa, Precio: 9513082.01 €
F. Lejeune - Puntos: 85.45, Posición: defensa, Precio: 9014216.64 €
A. Witsel - Puntos: 83.40, Posición: defensa, Precio: 9582015.82 €
D. Rico - Puntos: 81.74, Posición: defensa, Precio: 10101160.99 €
M. Mármol - Puntos: 69.39, Posición: defensa, Precio: 8004889.30 €
Y. Berchiche - Puntos: 69.39, Posición: defensa, Precio: 8004889.30 €
R. Le Normand - Puntos: 67.80, Posición: defensa, Precio: 8441801.20 €
M. Diakhaby - Puntos: 58.98, Posición: defensa, Precio: 7808851.99 €
S. Cardona - Puntos: 58.90, Posición: defensa, Precio: 5968691.32 €
Datos de defensas exportados a sugerencia_defensas.csv


In [22]:
top_10_mediocentros = resultados[resultados['Posicion'] == 'mediocentro'].sort_values(by='Puntos_Predichos', ascending=False).head(10)

print("\n10 mediocentros con mayor puntaje predicho:")
for i, row in top_10_mediocentros.iterrows():
    print(f"{row['Nombre']} - Puntos: {row['Puntos_Predichos']:.2f}, Posición: {row['Posicion']}, Precio: {row['Precio_Predicho']:.2f} €")

top_10_mediocentros = top_10_mediocentros.rename(columns={'Puntos_Predichos': 'Puntos', 'Precio_Predicho': 'Precio'})
top_10_mediocentros.to_csv('sugerencia_mediocentros.csv', index=False)
print("Datos de mediocentros exportados a sugerencia_mediocentros.csv")


10 mediocentros con mayor puntaje predicho:
J. Bellingham - Puntos: 124.86, Posición: mediocentro, Precio: 16379080.22 €
N. Williams - Puntos: 122.55, Posición: mediocentro, Precio: 17295089.78 €
F. Valverde - Puntos: 86.77, Posición: mediocentro, Precio: 10909847.38 €
O. Sancet - Puntos: 85.71, Posición: mediocentro, Precio: 9290414.63 €
R. De Paul - Puntos: 85.60, Posición: mediocentro, Precio: 9466325.91 €
J. Guerra - Puntos: 84.95, Posición: mediocentro, Precio: 10353177.22 €
T. Kroos - Puntos: 84.50, Posición: mediocentro, Precio: 11500140.77 €
V. Tsygankov - Puntos: 80.58, Posición: mediocentro, Precio: 10090350.59 €
Y. Herrera - Puntos: 78.97, Posición: mediocentro, Precio: 10290106.38 €
I. Martín - Puntos: 78.24, Posición: mediocentro, Precio: 8682085.93 €
Datos de mediocentros exportados a sugerencia_mediocentros.csv


In [23]:
top_10_porteros = resultados[resultados['Posicion'] == 'portero'].sort_values(by='Puntos_Predichos', ascending=False).head(10)

print("\n10 porteros con mayor puntaje predicho:")
for i, row in top_10_porteros.iterrows():
    print(f"{row['Nombre']} - Puntos: {row['Puntos_Predichos']:.2f}, Posición: {row['Posicion']},Precio: {row['Precio_Predicho']:.2f} €")

top_10_porteros = top_10_porteros.rename(columns={'Puntos_Predichos': 'Puntos', 'Precio_Predicho': 'Precio'})
top_10_porteros.to_csv('sugerencia_porteros.csv', index=False)
print("Datos de porteros exportados a sugerencia_porteros.csv")


10 porteros con mayor puntaje predicho:
V. Guaita - Puntos: 45.44, Posición: portero,Precio: 4291904.16 €
R. Fernández - Puntos: 41.49, Posición: portero,Precio: 1771496.53 €
A. Ferreira - Puntos: 35.60, Posición: portero,Precio: 1559934.49 €
D. Mariño - Puntos: 27.22, Posición: portero,Precio: 543815.69 €
U. Marrero - Puntos: 5.66, Posición: portero,Precio: 253212.05 €
A. López - Puntos: 0.62, Posición: portero,Precio: 791848.33 €
J. Doménech - Puntos: 0.00, Posición: portero,Precio: 221876.42 €
T. Courtois - Puntos: 0.00, Posición: portero,Precio: 1456785.91 €
P. Reina - Puntos: 0.00, Posición: portero,Precio: 203433.20 €
I. Grbić - Puntos: 0.00, Posición: portero,Precio: 189976.10 €
Datos de porteros exportados a sugerencia_porteros.csv
